# 人臉辨識

## OpenCV 的 Haar 人臉辨識

In [ ]:
!gdown '1VF-WVHOwfZYijEsAEF7ufNtuNUFqyIDJ' --output face_images.zip
!unzip face_images.zip

In [ ]:
!gdown '1IcRnaWQNgI5ukoeA_XqIxgIgA2o663N8' --output haar.zip
!unzip haar.zip

### webcam scan face

In [ ]:
# @title take photo with webcam
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

### upload image to analyse

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  img, text = haar_face(filename)
  print (text)
  plt.imshow(img)
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

### phone detect face

In [ ]:
def hdetect(path):
  img, text = haar_face(path)
  return text, img

import gradio as gr

demo = gr.Interface(hdetect, inputs = gr.Image(type = 'filepath', sources = 'webcam'), outputs = ['text', 'image'])
demo.launch(share = True)

In [ ]:
demo.close()

### find face

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Function to detect faces and draw rectangles around them
def haar_face(image_path):
    # Load the face detection model
    face_cascade = cv2.CascadeClassifier('/content/haar/haarcascade_frontalface_alt.xml')
    if face_cascade.empty():
        print('[找人臉]模型載入失敗.')
        return None, '模型載入失敗'

    print('[找人臉]模型載入成功.')

    # Read the image
    img = cv2.imread(image_path)
    if img is None:
        print('無法讀取圖像文件。')
        return None, '無法讀取圖像文件'

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect faces
    faces = face_cascade.detectMultiScale(img_rgb, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    print(f'Haar 找到相片中有 {len(faces)} 人.')

    # Draw rectangles around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (255, 0, 0), 2)

    return img_rgb, f'找到 {len(faces)} 人'

# Read and display the image
image_path = '/content/face_images/face1.jpg'
img, text = haar_face(image_path)
if img is not None:
    plt.imshow(img)
    plt.title(text)
    plt.axis('off')
    plt.show()
else:
    print(text)




In [ ]:
# @title 數人頭....請輸入相片檔名
圖檔 = 'face1.jpg'  # @param
path = '/content/face_images/' + 圖檔
img, text = haar_face(path)
print(text)
if img is not None:
    plt.imshow(img)
    plt.title(text)
    plt.axis('off')
    plt.show()

### find smiles

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load the smile detection model
找笑臉 = cv2.CascadeClassifier('/content/haar/haarcascade_smile.xml')
if 找笑臉.empty():
    print('[找笑臉]模型載入失敗.')
else:
    print('[找笑臉]模型載入成功.')

# Function to find and highlight smiles in an image
def find_smile(path, sf=2.5, minN=20):
    # Read the image
    img = cv2.imread(path)
    if img is None:
        print('無法讀取圖像文件。')
        return None

    # Convert the image to RGB format
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect smiles
    smiles = 找笑臉.detectMultiScale(img_rgb, scaleFactor=sf, minNeighbors=minN)
    print(f'找到 {len(smiles)} 個笑臉.')

    # Draw rectangles around the smiles
    for (x, y, w, h) in smiles:
        cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return img_rgb

# File path to the image
filename = '/content/face_images/face3_smile.jpg'

# Call the function to find smiles
result_img = find_smile(filename, 2.5, 20)

# Display the result
if result_img is not None:
    plt.imshow(result_img)
    plt.title('Detected Smiles')
    plt.axis('off')
    plt.show()


### 找眼睛

In [ ]:
# 讀入 [找眼睛] 模型
找眼睛 = cv2.CascadeClassifier('/content/haar/haarcascade_eye_tree_eyeglasses.xml')

# 把找到的眼睛畫出來
def find_eyes(path, sf = 1.1, minN = 3):
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  eyes = 找眼睛.detectMultiScale(img, scaleFactor = sf, minNeighbors = minN)
  for eye in eyes:
    x,y, w, h = eye
    x2, y2 = x + w, y + h
    cv2.rectangle(img, (x, y), (x2, y2), (0, 0, 255, 0), 5)
  return img

filename='/content/face_images/face3_smile.jpg'
find_eyes(filename)


### find all

In [ ]:

#find eyes, smiles, faces
def find_all(path, sf = 1.1, minN = 30):
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  faces = 找人臉.detectMultiScale(img)
  smiles = 找笑臉.detectMultiScale(img, scaleFactor = sf, minNeighbors = minN)
  eyes = 找眼睛.detectMultiScale(img)
  for face in faces:
    x, y, w, h = face
    x2, y2 = x + w, y + h
    cv2.rectangle(img, (x, y), (x2, y2), (255, 0, 0, 0), 5)
  for smile in smiles:
    x, y, w, h = smile
    x2, y2 = x + w, y + h
    cv2.rectangle(img, (x, y), (x2, y2), (0, 255, 0, 0), 5)
  for eye in eyes:
    x, y, w, h = eye
    x2, y2 = x + w, y + h
    cv2.rectangle(img, (x, y), (x2, y2), (0, 0, 255, 0), 5)
  return img

filename='/content/face_images/face3_smile.jpg'
find_all(filename, 2.5, 30)

### MTCNN 人臉偵測

In [ ]:
#加載需要的模組
from mtcnn import MTCNN
from matplotlib.patches import Rectangle, Circle
import matplotlib.pyplot as plt

# 用plt讀入圖片
img = plt.imread('/content/face_images/face1.jpg')
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# @title 建立偵測器
detector = MTCNN()
#偵測臉部
faces = detector.detect_faces(img)
faces

In [ ]:
# @title 計算相片中的人數
print(f'MTCNN 找到相片中有 {len(faces)}  個人')

faces[0]

In [ ]:
faces[0]['box']

In [ ]:
faces[0]['keypoints']['left_eye']

### 臉部加框與特徵點, 顯示偵測器的結果

In [ ]:
#臉部加框
plt.figure(figsize=(12,12))
ax = plt.gca()
for face in faces:
  #找出框的座標
  x, y, w, h = face['box']
  rect = Rectangle((x, y), w, h, fill = False, color = 'red')
  ax.add_patch(rect)
  #特徵點-左眼, 左嘴角, 右嘴角, nose, 右眼
  for key, value in face['keypoints'].items():
    dot = Circle(value, radius = 2, color = 'green')
    ax.add_patch(dot)

plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# 數人頭函式
from mtcnn import MTCNN
from matplotlib.patches import Rectangle,Circle
import matplotlib.pyplot as plt

detector = MTCNN()

def find_face(filename):
  img = plt.imread(filename)
  faces = detector.detect_faces(img)
  print(f'MTCNN找到相片中有 {len(faces)} 個人')
  ax = plt.gca()
  for face in faces:
    x, y, w, h = face['box']
    rect = Rectangle((x,y),w,h,fill=False,color='green')
    ax.add_patch(rect)
    for key,value in face['keypoints'].items():
      dot = Circle(value, radius=2, color='red')
      ax.add_patch(dot)
  plt.imshow(img)
  plt.axis('off')
  plt.show()

In [ ]:
filename='/content/face_images/face2.jpg'
find_face(filename)

In [ ]:
# @title 數人頭大PK....請輸入相片檔名
圖檔 = 'people2.jpg'# @param
path = '/content/face_images/' + 圖檔
img, text = haar_face(path)
find_face(path)
print(text)
plt.imshow(img)

In [ ]:
# @title webcamera
from IPython.display import Image
try:
  filename = take_photo()
  find_face(filename)
except Exception as err:
  print(str(err))

### 將每張臉截圖, 顯示出來

In [ ]:
# 把找到的臉截圖, 顯示出來
def cap_face(path):
  img = plt.imread(path)
  faces = detector.detect_faces(img)
  ax = plt.gca()
  for i, face in enumerate(faces):
    x, y, w, h = face['box']
    x2, y2 = x + w, y + h
    plt.subplot(1, len(faces), i + 1)
    plt.axis('off')
    plt.imshow(img[y:y2, x:x2])

plt.show()

cap_face('/content/face_images/people1.jpg')

## Deepface模組：人臉特徵分析工具

https://github.com/serengil/deepface

In [ ]:
!pip install -q deepface
!pip show deepface

Deepface 是一個用於 python 的輕量級人臉識別和人臉屬性分析（年齡、性別、情感和種族）框架。它是一個混合人臉識別框架，包含最先進的模型：VGG-Face, Google FaceNet, OpenFace, Facebook DeepFace, DeepID, ArcFace, Dlib and SFace.

實驗表明，人類在面部識別任務上的準確率達到 97.53%，而這些模型已經達到並通過了該準確度水平。

• 匯入模組：
```
from deepface import DeepFace
```
models: VGG-Face, Google FaceNet, OpenFace, Facebook DeepFace, DeepID, ArcFace, Dlib and SFace.

 VGG-Face, FaceNet, OpenFace, DeepFace, DeepID, ArcFace, Dlib, SFace and GhostFaceNet.

In [ ]:
#匯入模組
from deepface import DeepFace
import matplotlib.pyplot as plt
import cv2
import numpy as np

秀出照片

In [ ]:
def show_image(*args):
    k = len(args)
    fig = plt.figure(figsize=(5*k, 5))
    for i, photo in enumerate(args):
        plt.subplot(1,k,i+1)
        plt.axis('off')
        plt.axis('equal')
        plt.imshow(cv2.cvtColor(photo, cv2.COLOR_BGR2RGB))

## 人臉偵測



模型名稱：opencv(預設)、retinaface、mtcnn、dlib、ssd。

 OpenCV, Ssd, Dlib, MtCnn, Faster MtCnn, RetinaFace, MediaPipe, Yolo, YuNet and CenterFace.

enforce_detection強迫偵測,當沒有找到會出現錯誤, 因為偵測到之後會做後續處理, 所以一定要偵測到才行.

回傳人臉, 會把臉切圖下來, 做後續臉的比對.

• 臉部偵測：
```
圖片變數 = DeepFace.extract_faces(img_path=圖片路徑,
                detector_backend=模型名稱,
                enforce_detection=布林值)
模型名稱：opencv(預設)、retinaface、mtcnn、dlib、ssd
```

In [ ]:
# 秀出一張相片
img = cv2.imread('/content/1.jpg')
show_image(img)

In [ ]:
# 模型名稱：opencv(預設)、retinaface、mtcnn、dlib、ssd
img = DeepFace.extract_faces('1.jpg', detector_backend = 'opencv', enforce_detection = False)
plt.imshow(img[0]['face']) #showing face area
img[0] #details of image 

In [ ]:
# 偵測出人臉, 並且把結果存起來
img = DeepFace.extract_faces('4.jpg', detector_backend = 'mtcnn', enforce_detection = False)
for i in range(len(img)):
  # 結果是0-1之間值
  img2 = (img[0]['face']).astype(np.float32) * 255
  # 調整BGR2RBG
  img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
  #儲存
  cv2.imwrite('detect.jpg' + str(i) + '.jpg', img2)

In [ ]:
show_image(cv2.imread('detect.jpg'))

偵測人臉並截圖

In [ ]:
def detectFace(path, model = 'opencv'):
  img = DeepFace.extract_faces(path, detector_backend = model, enforce_detection = False)
  img2 = img[0]['face']
  return img2

img = detectFace('1.jpg', 'opencv')
plt.imshow(img)

### camera capture


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo('cap.jpg')
  img = detectFace(filename, 'dlib')
  plt.imshow(img)
  plt.axis('off')
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

## 用gradio做人臉偵測網路服務應用

In [ ]:
!pip install gradio --quiet

In [ ]:
# 偵測人臉且截圖
def detectFace2(path, model='opencv'):
  img = DeepFace.extract_faces(path,
                             detector_backend= model,
                             enforce_detection=False)
  img2 = (img[0]['face']).astype(np.float32)
  # 調整BGR2RBG
  img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
  return img2

### 人臉偵測遊戲

In [ ]:
import gradio as gr

import gradio as gr

demo = gr.Interface(
    detectFace2,
    inputs=['image',
            gr.Radio(["opencv", "mtcnn", "dlib"], label="模型", info="請選擇一個模型")],
    outputs=['image'],
    title='人臉偵測遊戲')

demo.launch(share=True)

In [ ]:
demo.close()

In [ ]:
img = detectFace('1.jpg', 'opencv')
show_image(img)

### 臉部驗證
判斷兩個人是否為同一個人

Deepface模組：人臉特徵分析工具2

• 臉部驗證：
```
驗證變數 = DeepFace.verify(img1_path=圖片路徑1,
              img2_path=圖片路徑2,
              model_name=驗證模型名稱,
              model=建立模型,
              detector_backend=偵測模型名稱,
              distance_metric=距離計算方式,
              enforce_detection=布林值)
```

相似度可以通過不同的度量來計算，例如餘弦相似度、歐氏距離和 L2 形式。默認配置使用餘弦相似度。


```
metrics = ["cosine", "euclidean", "euclidean_l2"]
```


metrics = ["cosine", "euclidean", "euclidean_l2"]

Deepface 是一個混合人臉識別包。它目前包含許多最先進的人臉識別模型：VGG-Face、Google FaceNet、OpenFace、Facebook DeepFace、DeepID、ArcFace和。默認配置使用 VGG-Face 模型


```
models = ["VGG-Face","Facenet","Facenet512","OpenFace","DeepFace","DeepID","ArcFace","Dlib","SFace","GhostFaceNet",]
```



In [ ]:
# 顯示兩張比對相片
file1 = '2.jpg'
file2 = '3.jpg'
show_image(cv2.imread(file1), cv2.imread(file2))

In [ ]:
# 臉部驗證, 第一次跑 DeepFace 要等 1 分鐘
result = DeepFace.verify(file1, file2,
                         model_name='VGG-Face',
                         enforce_detection=False)
result



```
{'verified': True,
 'distance': 0.2730201547529547,
 'threshold': 0.4,
 'model': 'VGG-Face',
 'detector_backend': 'opencv',
 'similarity_metric': 'cosine',
 'facial_areas': {'img1': {'x': 506,
   'y': 234,
   'w': 211,
   'h': 211,
   'left_eye': (642, 308),
   'right_eye': (572, 322)},
  'img2': {'x': 297,
   'y': 74,
   'w': 140,
   'h': 140,
   'left_eye': (391, 125),
   'right_eye': (342, 131)}},
 'time': 20.1}
```
'distance': 愈小愈相似.

- distance <= threshold(0.4) 同一人
- distance > threshold 不同人


In [ ]:
# 判定結果在 'verified': True 同一個人, False 不同人
if result['verified']:
  print('same person') 
else:
  print('not the same person')

In [ ]:
# 臉部驗證
def face_verify(path1, path2, threshold = 0.6, model = 'VGG-Face'):
  ret = DeepFace.verify(path1, path2, model_name = model)
  if ret['distance'] <= threshold:
    text = 'same person'
  else:
    text = 'not the same person'
  return text

face_verify('2.jpg', '3.jpg')

In [ ]:
demo = gr.Interface(fn = face_verify, inputs = ['image', 'image'], outputs = ['label'], title = 'same one?')
demo.launch(share=True)

In [ ]:
demo.close()

### 搜尋人臉: 人臉比對

• 搜尋人臉：
```
搜尋變數 = DeepFace.find(img_path=圖片路徑,
             db_path=圖片資料夾目錄路徑,
             model_name=驗證模型名稱,
             model=建立模型,
             detector_backend=偵測模型名稱,
             distance_metric=距離計算方式,
             enforce_detection=布林值)
```
注意：臉圖片資料庫目錄中增減圖片時，

記得要先刪除
<representations_vgg_face.pkl>檔案，

讓Deepface重新訓練產生新的人臉圖片資料庫模型檔案。

#### 人臉相片打卡break down

In [ ]:
# 載入需要用的模組
import pandas as pd
import numpy as np

member 內有數個人.

In [ ]:
member = {'Jay_Chou':'周杰倫','Jolin_Tsai':'蔡依琳','Lin_Chi-Ling':'林志玲',
      'Nick_Chou':'周湯豪','Richie_Jen':'任賢齊','New':'新員工'}

# 找 member 裡的人
file1 = '1.jpg'
show_image(cv2.imread(file1))

db = 'member'
df = DeepFace.find(file1, db, model_name = 'VGG-Face', enforce_detection = False)
df

In [ ]:
df[0]['identity']

In [ ]:
df[0]['identity'][0]

In [ ]:
df[0]['identity'][0].split('/')

In [ ]:
df[0]['identity'][0].split('/')[1]

In [ ]:
member

In [ ]:
s = df[0]['identity'][0].split('/')[1]
member[s]



```
[                 identity                                      hash  target_x  \
 0  member/Nick_Chou/2.png  d939dfc23ad656c142936d93458c1a78ed37f9e5        83   
 1   member/Jay_Chou/1.jpg  748844de37a281e6950ade15880e45732ecaaaee       236   

    target_y  target_w  target_h  source_x  source_y  source_w  source_h  \
 0        93       204       204       506       234       211       211   
 1        82       133       133       506       234       211       211   

    threshold  distance  
 0       0.68  0.372531  
 1       0.68  0.542073  ]
```



In [ ]:
df[0]['distance']

distance是距離,愈小愈相似

In [ ]:
#尋找單一相同人臉 , 看有多少人符合條件 距離小於0.21
c = np.sum(df[0]['distance'] <= 0.5)
print(f'找到相同人臉有 {c} 張相片.')

# 找出比對出的員工姓名
if c>0:
  s =  df[0]['identity'][0].split('/')[1] #取出檔名
  print(f'{member[s]} 早安, 上班打卡成功.')
else:
  print('非本公司員工, 請至警衛室登記！')

### 尋找所有相同人臉
把有我的相片找出來, 上面程式整理
Dlib 要有GPU

In [ ]:
#尋找所有相同人臉, 把有這個人的相片全部找出來
# 人臉相片打卡完整程式
from deepface import DeepFace
import pandas as pd
import numpy as np

def find_photos(path, db = 'member', threshold = 0.4, model = 'VGG-Face'):
  df = DeepFace.find(path, db, model_name = model, enforce_detection = False)
  c = np.sum(df[0]['distance'] <= threshold)
  if c>0:
    print(f'找到 {c} 張相片.')
    for i in range(c):
      s = df[0]['identity'][i]
      print(f'第 {i+1} 張相片的路徑檔名是 {s}')
  else:
    print('沒有你的相片.')

find_photos('2.jpg', threshold = 0.6)

在member1會建立representations_vgg_face.pkl,

程式執行時會檢查這個檔案是否存在, 如果不存在就會用相片去建立這個檔案, 它是圖片特徵資料庫.

如果已經有這個檔案, 就直接去用這個檔案去比對, 節省處理時間.

**臉圖片資料庫目錄中增減圖片時，記得要先刪除 <representations_vgg_face.pkl>檔案，讓Deepface重新訓練產生新的人臉圖片資料 庫模型檔案。**

### 叫用webcam (記得執行這隻程式)

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# webcam 人臉登入程式
from IPython.display import Image
try:
  filename = take_photo('cap.jpg')
  text = face_checkin(filename)
  print(text)
except Exception as err:
  print(f'發生錯誤, {str(err)}')

#### 有新進員工時的人臉登入, 該怎麼做呢?
把新進員工相片存到 member 資料夾.

1. 建立 New 資料夾
2. 拍好的照片會存到這個資料夾
3. member 字典要加上 'New':'新進員工'

用程式來建立新員工相片資料夾.

```
import os
  
# 先檢查資料夾是否存在.
if not os.path.exists("./member/New"):       
    # 如果不存在就建立它
    os.makedirs("./member/New")
```



In [ ]:
import os

# 先檢查資料夾是否存在.
if not os.path.exists("./member/New"):
    # 如果不存在就建立它
    os.makedirs("./member/New")

把新進員工相片存到 member1 資料夾.

In [ ]:
# 把新進員工相片存到 member 資料夾
from IPython.display import Image
try:
  filename = take_photo('member/New/new.jpg')
  display(Image(filename))
except Exception as err:
  print(str(err))

# 在member 字典加上 'New':'新進員工'
member = {'Jay_Chou':'周杰倫','Jolin_Tsai':'蔡依琳','Lin_Chi-Ling':'林志玲',
      'Nick_Chou':'周湯豪','Richie_Jen':'任賢齊','New':'新進員工'}

**注意**
臉圖片資料庫目錄中增減圖片時，

記得要先刪除 <*.pkl> 檔案，
重新訓練產生新的人臉圖片資料 庫模型檔案。

In [ ]:
!rm ./member/*.pkl

In [ ]:
# 加入新進員工後的人臉登入
from IPython.display import Image
try:
  filename = take_photo('cap.jpg')
  text = face_checkin(filename)
  print(text)
except Exception as err:
  print(f'發生錯誤, {str(err)}')

#### 新員工註冊

In [ ]:
def regMeb2(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  path = 'member/New/new.jpg'
  cv2.imwrite(path, img)
  return 'sucessful registration'

import gradio as gr
demo = gr.Interface(regMeb2, 'image', 'text', title = 'registration system')

In [ ]:
demo.launch(share = True)

In [ ]:
demo.shutdown()

In [ ]:
demo.shutdown()

In [ ]:
demo = gr.Interface(
    face_checkin,
    inputs=gr.Image(type='filepath', label='拍照'),
    outputs=gr.Label()
)
demo.launch(share=True)

### 應用：AI 面相館
• 人臉屬性分析：
```
屬性變數 = DeepFace.analyze(img_path=圖片路徑,
               actions=屬性串列,
               detector_backend=偵測模型名稱,
               enforce_detection=布林值)
```

In [ ]:
# 人臉屬性分析, 跑要1分鐘. actions=屬性串列
# actions=['age', 'gender', 'race', 'emotion']
attri = DeepFace.analyze('3.jpg', actions = ['age', 'gender', 'race', 'emotion'], enforce_detection = False)
attri

In [ ]:
print('年齡', attri[0]['age'])
print('性別', attri[0]['dominant_gender'])
print('種族', attri[0]['dominant_race'])
print('情緒', attri[0]['dominant_emotion'])
show_image(cv2.imread('3.jpg'))

偵測結果用中文顯示

In [ ]:
labels = {'angry':'生氣', 'disgust':'厭惡', 'fear':'恐懼',
      'happy':'開心', 'neutral':'沒什麼特別表情',
      'sad':'悲傷', 'surprise':'吃驚',
      'Man':'男', 'Woman':'女',
      'asian':'亞洲', 'black':'黑', 'indian':'印弟安',
      'latino hispanic':'拉丁美洲 (西班牙裔)',
      'middle eastern':'中東', 'white':'白'}

def show_info(obj):
    age = obj['age']
    emotion = labels[obj['dominant_emotion']]
    race = labels[obj['dominant_race']]
    gender = labels[obj['dominant_gender']]
    text = f"這是一位 {age} 歲的{race}人{gender}子, 他/她感覺是{emotion}的。"
    #print(text)
    return text

In [ ]:
# 用一句中文來顯示 AI 分析結果
show_info(attri[0])

用webcam來偵測人臉屬性

In [ ]:
from IPython.display import Image
try:
  filename = take_photo('cap.jpg')
  display(Image(filename))
  # 人臉屬性分析, actions=屬性串列
  attri = DeepFace.analyze(filename,
                           actions=['age','gender','race','emotion'],
                           enforce_detection=False)
  text = show_info(attri[0])
  print(text)
except Exception as err:
  print(str(err))

### AI 面相館(情緒分析)

#### 偵測結果用中文顯示

In [ ]:
labels = {'angry':'生氣', 'disgust':'厭惡', 'fear':'恐懼',
      'happy':'開心', 'neutral':'沒什麼特別表情',
      'sad':'悲傷', 'surprise':'吃驚',
      'Man':'男', 'Woman':'女',
      'asian':'亞洲', 'black':'黑', 'indian':'印弟安',
      'latino hispanic':'拉丁美洲 (西班牙裔)',
      'middle eastern':'中東', 'white':'白'}

def show_info(obj):
    age = obj['age']
    emotion = labels[obj['dominant_emotion']]
    race = labels[obj['dominant_race']]
    gender = labels[obj['dominant_gender']]
    text = f"這是一位 {age} 歲的{race}人{gender}子, 他感覺是{emotion}的。"
    #print(text)
    return text

In [ ]:
def AI面相大師(path):
  obj = DeepFace.analyze(path,
               actions=['age','gender','race','emotion'],
               enforce_detection=False)
  label = show_info(obj[0])
  return label

In [ ]:
AI面相大師('1.jpg')

In [ ]:
import gradio as gr

demo = gr.Interface(
    AI面相大師,
    inputs=gr.Image(type='filepath', label='相片'),
    outputs='label',
    title = 'AI面相算命館',
    description='# 不準不用錢!!!',
    examples=['2.jpg','3.jpg']
)
demo.launch(share=True)

In [ ]:

# 顯示兩張比對相片
file1 = '2.jpg'
file2 = '3.jpg'
show_image(cv2.imread(file1), cv2.imread(file2))
# 臉部驗證, 第一次跑 DeepFace 要等 1 分鐘
result = DeepFace.verify(file1, file2,
                         model_name='VGG-Face',
                         enforce_detection=False)
result


```
{'verified': True,
 'distance': 0.2730201547529547,
 'threshold': 0.4,
 'model': 'VGG-Face',
 'detector_backend': 'opencv',
 'similarity_metric': 'cosine',
 'facial_areas': {'img1': {'x': 506,
   'y': 234,
   'w': 211,
   'h': 211,
   'left_eye': (642, 308),
   'right_eye': (572, 322)},
  'img2': {'x': 297,
   'y': 74,
   'w': 140,
   'h': 140,
   'left_eye': (391, 125),
   'right_eye': (342, 131)}},
 'time': 20.1}
```
'distance': 愈小愈相似.

- distance <= threshold(0.4) 同一人
- distance > threshold 不同人

# 判定結果在 'verified': True 同一個人, False 不同人
if result['verified']:
  print('same person') 
else:
  print('not the same person')
# 臉部驗證
def face_verify(path1, path2, threshold = 0.6, model = 'VGG-Face'):
  ret = DeepFace.verify(path1, path2, model_name = model)
  if ret['distance'] <= threshold:
    text = 'same person'
  else:
    text = 'not the same person'
  return text

face_verify('2.jpg', '3.jpg')
demo = gr.Interface(fn = face_verify, inputs = ['image', 'image'], outputs = ['label'], title = 'same one?')
demo.launch(share=True)
demo.close()
### 搜尋人臉: 人臉比對
• 搜尋人臉：
```
搜尋變數 = DeepFace.find(img_path=圖片路徑,
             db_path=圖片資料夾目錄路徑,
             model_name=驗證模型名稱,
             model=建立模型,
             detector_backend=偵測模型名稱,
             distance_metric=距離計算方式,
             enforce_detection=布林值)
```
注意：臉圖片資料庫目錄中增減圖片時，

記得要先刪除
<representations_vgg_face.pkl>檔案，

讓Deepface重新訓練產生新的人臉圖片資料庫模型檔案。
#### 人臉相片打卡break down
# 載入需要用的模組
import pandas as pd
import numpy as np
member 內有數個人.
member = {'Jay_Chou':'周杰倫','Jolin_Tsai':'蔡依琳','Lin_Chi-Ling':'林志玲',
      'Nick_Chou':'周湯豪','Richie_Jen':'任賢齊','New':'新員工'}

# 找 member 裡的人
file1 = '1.jpg'
show_image(cv2.imread(file1))

db = 'member'
df = DeepFace.find(file1, db, model_name = 'VGG-Face', enforce_detection = False)
df
df[0]['identity']
df[0]['identity'][0]
df[0]['identity'][0].split('/')
df[0]['identity'][0].split('/')[1]
member
s = df[0]['identity'][0].split('/')[1]
member[s]


```
[                 identity                                      hash  target_x  \
 0  member/Nick_Chou/2.png  d939dfc23ad656c142936d93458c1a78ed37f9e5        83   
 1   member/Jay_Chou/1.jpg  748844de37a281e6950ade15880e45732ecaaaee       236   

    target_y  target_w  target_h  source_x  source_y  source_w  source_h  \
 0        93       204       204       506       234       211       211   
 1        82       133       133       506       234       211       211   

    threshold  distance  
 0       0.68  0.372531  
 1       0.68  0.542073  ]
```


df[0]['distance']
distance是距離,愈小愈相似
#尋找單一相同人臉 , 看有多少人符合條件 距離小於0.21
c = np.sum(df[0]['distance'] <= 0.5)
print(f'找到相同人臉有 {c} 張相片.')

# 找出比對出的員工姓名
if c>0:
  s =  df[0]['identity'][0].split('/')[1] #取出檔名
  print(f'{member[s]} 早安, 上班打卡成功.')
else:
  print('非本公司員工, 請至警衛室登記！')
### 尋找所有相同人臉
把有我的相片找出來, 上面程式整理
Dlib 要有GPU
#尋找所有相同人臉, 把有這個人的相片全部找出來
# 人臉相片打卡完整程式
from deepface import DeepFace
import pandas as pd
import numpy as np

def find_photos(path, db = 'member', threshold = 0.4, model = 'VGG-Face'):
  df = DeepFace.find(path, db, model_name = model, enforce_detection = False)
  c = np.sum(df[0]['distance'] <= threshold)
  if c>0:
    print(f'找到 {c} 張相片.')
    for i in range(c):
      s = df[0]['identity'][i]
      print(f'第 {i+1} 張相片的路徑檔名是 {s}')
  else:
    print('沒有你的相片.')

find_photos('2.jpg', threshold = 0.6)
在member1會建立representations_vgg_face.pkl,

程式執行時會檢查這個檔案是否存在, 如果不存在就會用相片去建立這個檔案, 它是圖片特徵資料庫.

如果已經有這個檔案, 就直接去用這個檔案去比對, 節省處理時間.

**臉圖片資料庫目錄中增減圖片時，記得要先刪除 <representations_vgg_face.pkl>檔案，讓Deepface重新訓練產生新的人臉圖片資料 庫模型檔案。**
### 叫用webcam (記得執行這隻程式)
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename
# webcam 人臉登入程式
from IPython.display import Image
try:
  filename = take_photo('cap.jpg')
  text = face_checkin(filename)
  print(text)
except Exception as err:
  print(f'發生錯誤, {str(err)}')
#### 有新進員工時的人臉登入, 該怎麼做呢?
把新進員工相片存到 member 資料夾.

1. 建立 New 資料夾
2. 拍好的照片會存到這個資料夾
3. member 字典要加上 'New':'新進員工'

用程式來建立新員工相片資料夾.

```
import os
  
# 先檢查資料夾是否存在.
if not os.path.exists("./member/New"):       
    # 如果不存在就建立它
    os.makedirs("./member/New")
```


import os

# 先檢查資料夾是否存在.
if not os.path.exists("./member/New"):
    # 如果不存在就建立它
    os.makedirs("./member/New")
把新進員工相片存到 member1 資料夾.
# 把新進員工相片存到 member 資料夾
from IPython.display import Image
try:
  filename = take_photo('member/New/new.jpg')
  display(Image(filename))
except Exception as err:
  print(str(err))

# 在member 字典加上 'New':'新進員工'
member = {'Jay_Chou':'周杰倫','Jolin_Tsai':'蔡依琳','Lin_Chi-Ling':'林志玲',
      'Nick_Chou':'周湯豪','Richie_Jen':'任賢齊','New':'新進員工'}
**注意**
臉圖片資料庫目錄中增減圖片時，

記得要先刪除 <*.pkl> 檔案，
重新訓練產生新的人臉圖片資料 庫模型檔案。
!rm ./member/*.pkl
# 加入新進員工後的人臉登入
from IPython.display import Image
try:
  filename = take_photo('cap.jpg')
  text = face_checkin(filename)
  print(text)
except Exception as err:
  print(f'發生錯誤, {str(err)}')
#### 新員工註冊
def regMeb2(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  path = 'member/New/new.jpg'
  cv2.imwrite(path, img)
  return 'sucessful registration'

import gradio as gr
demo = gr.Interface(regMeb2, 'image', 'text', title = 'registration system')
demo.launch(share = True)
demo.shutdown()
demo.shutdown()
demo = gr.Interface(
    face_checkin,
    inputs=gr.Image(type='filepath', label='拍照'),
    outputs=gr.Label()
)
demo.launch(share=True)
### 應用：AI 面相館
• 人臉屬性分析：
```
屬性變數 = DeepFace.analyze(img_path=圖片路徑,
               actions=屬性串列,
               detector_backend=偵測模型名稱,
               enforce_detection=布林值)
```
# 人臉屬性分析, 跑要1分鐘. actions=屬性串列
# actions=['age', 'gender', 'race', 'emotion']
attri = DeepFace.analyze('3.jpg', actions = ['age', 'gender', 'race', 'emotion'], enforce_detection = False)
attri
print('年齡', attri[0]['age'])
print('性別', attri[0]['dominant_gender'])
print('種族', attri[0]['dominant_race'])
print('情緒', attri[0]['dominant_emotion'])
show_image(cv2.imread('3.jpg'))
偵測結果用中文顯示
labels = {'angry':'生氣', 'disgust':'厭惡', 'fear':'恐懼',
      'happy':'開心', 'neutral':'沒什麼特別表情',
      'sad':'悲傷', 'surprise':'吃驚',
      'Man':'男', 'Woman':'女',
      'asian':'亞洲', 'black':'黑', 'indian':'印弟安',
      'latino hispanic':'拉丁美洲 (西班牙裔)',
      'middle eastern':'中東', 'white':'白'}

def show_info(obj):
    age = obj['age']
    emotion = labels[obj['dominant_emotion']]
    race = labels[obj['dominant_race']]
    gender = labels[obj['dominant_gender']]
    text = f"這是一位 {age} 歲的{race}人{gender}子, 他/她感覺是{emotion}的。"
    #print(text)
    return text
# 用一句中文來顯示 AI 分析結果
show_info(attri[0])
用webcam來偵測人臉屬性
from IPython.display import Image
try:
  filename = take_photo('cap.jpg')
  display(Image(filename))
  # 人臉屬性分析, actions=屬性串列
  attri = DeepFace.analyze(filename,
                           actions=['age','gender','race','emotion'],
                           enforce_detection=False)
  text = show_info(attri[0])
  print(text)
except Exception as err:
  print(str(err))
### AI 面相館(情緒分析)
#### 偵測結果用中文顯示
labels = {'angry':'生氣', 'disgust':'厭惡', 'fear':'恐懼',
      'happy':'開心', 'neutral':'沒什麼特別表情',
      'sad':'悲傷', 'surprise':'吃驚',
      'Man':'男', 'Woman':'女',
      'asian':'亞洲', 'black':'黑', 'indian':'印弟安',
      'latino hispanic':'拉丁美洲 (西班牙裔)',
      'middle eastern':'中東', 'white':'白'}

def show_info(obj):
    age = obj['age']
    emotion = labels[obj['dominant_emotion']]
    race = labels[obj['dominant_race']]
    gender = labels[obj['dominant_gender']]
    text = f"這是一位 {age} 歲的{race}人{gender}子, 他感覺是{emotion}的。"
    #print(text)
    return text
def AI面相大師(path):
  obj = DeepFace.analyze(path,
               actions=['age','gender','race','emotion'],
               enforce_detection=False)
  label = show_info(obj[0])
  return label
AI面相大師('1.jpg')
import gradio as gr

demo = gr.Interface(
    AI面相大師,
    inputs=gr.Image(type='filepath', label='相片'),
    outputs='label',
    title = 'AI面相算命館',
    description='# 不準不用錢!!!',
    examples=['2.jpg','3.jpg']
)
demo.launch(share=True)